![img](https://www.kdnuggets.com/wp-content/uploads/text-preprocessing-framework-2.png)

In [1]:
! pip install contractions

In [2]:
! pip install inflect

In [3]:
! pip install wget

In [199]:
import pandas as pd
import numpy as np
import nltk.data
from nltk import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import contractions
import inflect
import re, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, LSTM, RepeatVector, TimeDistributed
from keras.utils import plot_model

#import my_functions

pd.set_option('max_colwidth', 250)

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Enric\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
STOPWORDS_ESP = list(set(stopwords.words('spanish')))
STOPWORDS_CAT = open('stopwords_catalan.txt', 'r', encoding= 'UTF-8').read().split()
STOPWORDS_ENG = list(set(stopwords.words('english')))
CUSTOM  = ['uf', '100st', '100t', '100u', 'pal', 'zzb21601xu', 'x2']

STOPWORDS_ALL = STOPWORDS_ESP + STOPWORDS_CAT + STOPWORDS_ENG + CUSTOM

In [5]:
import wget

print('Beginning file download with wget module')

url = 'https://raw.githubusercontent.com/michmech/lemmatization-lists/master/lemmatization-es.txt'  
#wget.download(url) 

Beginning file download with wget module


In [6]:
lemmatizer = {}
with open("lemmatization-es.txt", encoding= 'UTF-8') as f:
    for line in f:
        try:
            (key, val) = line.split()
        except:
            pass
        lemmatizer[str(key)] = val

In [7]:
lemmatizer_inv = {v: k for k, v in lemmatizer.items()}

In [8]:
#lemmatizer_inv

In [9]:
data = pd.read_csv('../02_Comprension de Datos/Descripciones204k_20190315.csv', sep = ';')

In [10]:
data.head()

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo masculino con textura de mujer
1,1060651400180,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo doble faz de mujer con cinturón a tono
2,1051056400107,Woman_El_Corte_Inglés,Moda Mujer Abrigos Abrigo largo de antelina de mujer Woman El Corte Inglés
3,1019350401147,Lloyd's,Moda Mujer Abrigos Chaqueta térmica de mujer Lloyds con efecto cortavientos
4,1019353400229,Lloyd's,Moda Mujer Abrigos Parka 100% algodón de mujer Lloyds con capucha


Creamos una lista de marcas para poder quitarlas del texto en caso de que aparezcan, eliminamos las marcas duplicadas para comprimir el tamaño de la lista y optimizar el proceso de bsuqueda

In [11]:
data.loc[199000]

id                                        1025942232486
brand                                          Fox_Home
text     Cine Series de Tv Futurama. 7ª Temporada (DVD)
Name: 199000, dtype: object

In [12]:
copia = data.copy()

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize_brands(words):
    words = str(words)
    words  = words.strip().split()
    words = to_lowercase(words)
    words = remove_punctuation(words)    
    words = list(dict.fromkeys(words)) #Remove duplicates
    words = " ".join(words)
    return words
    
copia['brand'] = copia['brand'].apply(normalize_brands)
brands = list(dict.fromkeys(copia['brand'].values))
brands.remove('marvel')
more_brands = ['tarocco', 'tucci', 'fjord', 'tommy', 'hilfiger', 'easy', 'wear', 'josma', 'jaipur', 'ralph', 'nemeziz']
for brand in more_brands:
    brands.append(brand)

In [13]:
len(brands)

5091

## Normalizacion

Con la normalización vamos a realizar una serie de tareas destinadas a poner todo el texto en un campo de juego nivelado: convertir todo el texto a la misma mayúscula o minúscula, eliminar la puntuación, convertir las cifras a sus equivalentes en palabras, etc. La normalización pone todas las palabras en pie de igualdad y permite que el procesamiento se realice de manera uniforme. Algunas de las técnicas que vamos a aplicar son:

- Borrar caracteres extraños
- Pasar todo el texto a minusculas
- Quitar simbolos de puntuacion ( . , &, !, ?, ¿, /, etc)
- Pasar numeros en digito a texto
- Quitar stopwords
- Stemming
- Lexematizar

**Importante:** Despues de la Nomralizacion trabajaremos a nivel de palabra o token en vez de a nivel de texto

In [14]:
ngrams = ['corte ingles', 'bob esponja', 'color azul', 'cristiano ronaldo', 'fc barcelona', 'lionel messi', 'real madrid',
         'azul oscuro', '1a temporada', '2a temporada', '3a temporada', '4a temporada', '5a temporada', '6a temporada',
         '7a temporada', '8a temporada', 'mrs increible', 'mr increible', 'fc barcelona']

In [15]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        #new_word = unicodedata.normalize('NFD', word).encode('ascii', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Remove all interger occurrences in list"""
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in STOPWORDS_ALL and len(word) > 1:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('spanish')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def remove_brands(words):
    """Remove brands from the text"""
    clean_words = []
    for word in words:
        if word not in brands:
            clean_words.append(word)
    return clean_words

def lemmatize(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmas = []
    for word in words:
        if word in lemmatizer_inv:
            lemmas.append(lemmatizer_inv[word])
        else: lemmas.append(word)
    return lemmas

def replace_ngrams(words):
    for gram in ngrams:
        if (gram in words):
            g = "_".join(gram.split())
            words = words.replace(gram, g)
    return words

def normalize(words):
    #if pandas == True:
    words  = words.strip().split()
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_brands(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    words = replace_numbers(words)
    words = remove_non_ascii(words)
    words = list(dict.fromkeys(words)) #Remove duplicates
    words = " ".join(words)
    words = replace_ngrams(words)
    return words


In [16]:
data_copy = data.copy()

In [17]:
%%time
#4min 4sec
data_copy['text'] = data_copy['text'].apply(normalize)

Wall time: 20min 42s


In [18]:
data_copy.head(5)

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,moda mujer abrigo masculino textura
1,1060651400180,Woman_Limited_El_Corte_Inglés,moda mujer abrigo doble faz cinturon tono
2,1051056400107,Woman_El_Corte_Inglés,moda mujer abrigo largo antelina woman corte_ingles
3,1019350401147,Lloyd's,moda mujer abrigo chaqueta termica efecto cortavientos
4,1019353400229,Lloyd's,moda mujer abrigo parka algodon capucha


In [19]:
data.head(5)

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo masculino con textura de mujer
1,1060651400180,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo doble faz de mujer con cinturón a tono
2,1051056400107,Woman_El_Corte_Inglés,Moda Mujer Abrigos Abrigo largo de antelina de mujer Woman El Corte Inglés
3,1019350401147,Lloyd's,Moda Mujer Abrigos Chaqueta térmica de mujer Lloyds con efecto cortavientos
4,1019353400229,Lloyd's,Moda Mujer Abrigos Parka 100% algodón de mujer Lloyds con capucha


In [20]:
#7000
print(data.loc[199000])
print(data_copy.loc[89000])

id                                        1025942232486
brand                                          Fox_Home
text     Cine Series de Tv Futurama. 7ª Temporada (DVD)
Name: 199000, dtype: object
id                                       1014970203072
brand                                Polo_Ralph_Lauren
text     moda mujer punto jersey lana polo cuello caja
Name: 89000, dtype: object


In [21]:
data_copy['text'].shape

(204812,)

In [22]:
maxw = 0
mean = 0
max_s = 0
for i, sentence in enumerate(data_copy['text']):
    mean += len(sentence.split())
    if maxw < len(sentence.split()):
        maxw = len(sentence.split())
        max_s = i
print('Max words in a sentence:' + ' '*10 + 'Mean words in a sentence:')
print("-" * 60)
print (f"{maxw} {' '*30} {mean/len(data_copy['text'])}")
print("-" * 60)
print (data_copy['text'][max_s])

Max words in a sentence:          Mean words in a sentence:
------------------------------------------------------------
23                                8.059176220143351
------------------------------------------------------------
libro salud bienestar dieta nutricion general detox sen sano dentro bello clave nutricional rutina diarias eliminar toxina forma saludable energetica nutritiva tapa blanda


In [23]:
data_copy.to_csv('Texto_PreProcesado.csv', sep = ';', index = False)

### N-gramas (CountVectorizer)

In [24]:
bag_of_words = nltk.word_tokenize(data_copy['text'].to_string())
bag_of_words = list(dict.fromkeys(bag_of_words))
print(bag_of_words[:10])
print(len(bag_of_words)) 
#126k productos = 2_063_845 palabras
#175k productos = 2_838_097 palabras
#204k productos = 3_070_773 palabras -> 1_878_308 20/03/2019 -> no dups = 257042, ngrams =257063 

['0', 'moda', 'mujer', 'abrigo', 'masculino', 'textura', '1', 'doble', 'faz', 'cinturon']
257064


In [25]:
corpus = [sent for sent in data_copy['text']]
corpus[0:5]

['moda mujer abrigo masculino textura',
 'moda mujer abrigo doble faz cinturon tono',
 'moda mujer abrigo largo antelina woman corte_ingles',
 'moda mujer abrigo chaqueta termica efecto cortavientos',
 'moda mujer abrigo parka algodon capucha']

In [26]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(corpus)

In [27]:
list(vectorizer.vocabulary_.keys())[0:10]

['moda',
 'mujer',
 'abrigo',
 'masculino',
 'textura',
 'moda mujer',
 'mujer abrigo',
 'abrigo masculino',
 'masculino textura',
 'doble']

---

## Word2Vec


In [28]:
EMBEDDING_DIM = 200

In [88]:
sentences = [word.split() for word in data_copy['text'].values]
sentences[:3]

[['moda', 'mujer', 'abrigo', 'masculino', 'textura'],
 ['moda', 'mujer', 'abrigo', 'doble', 'faz', 'cinturon', 'tono'],
 ['moda', 'mujer', 'abrigo', 'largo', 'antelina', 'woman', 'corte_ingles']]

In [30]:
len(sentences)

204812

In [31]:
%%time
model = Word2Vec(sentences, workers = 3, min_count=5, window = 10, size = EMBEDDING_DIM)
model.train(sentences, total_examples=len(sentences), epochs=50)

Wall time: 2min 48s


In [32]:
model.save("word2vec_model_v2")

In [33]:
#model = Word2Vec.load("word2vec_model_v1")
wl = 'sandalia'
model.wv.most_similar (positive = wl)
#model.wv.most_similar_cosmul(positive = ['disfraz', 'abrigo'])

[('alpargata', 0.6205319762229919),
 ('botin', 0.6136024594306946),
 ('salon', 0.5804810523986816),
 ('mocasin', 0.5511438250541687),
 ('chancla', 0.5500357151031494),
 ('mercedita', 0.5439091920852661),
 ('bota', 0.5414243340492249),
 ('pepito', 0.5377013087272644),
 ('nautico', 0.5359429717063904),
 ('deportivas', 0.5105116963386536)]

In [34]:
def similar_products(text):
    text = normalize(text)
    list_text = text.split()
    most_similar = model.wv.most_similar_cosmul(positive = list_text)
    
    return most_similar
    
similar_products('Silla de paseo')

[('carrycot', 0.46414005756378174),
 ('cambiador', 0.4590770900249481),
 ('portabebes', 0.44152915477752686),
 ('maternal', 0.4405578374862671),
 ('trona', 0.43963176012039185),
 ('seat', 0.43299543857574463),
 ('yaki', 0.4328617453575134),
 ('lux', 0.4298858642578125),
 ('capazo', 0.42714881896972656),
 ('modular', 0.42647257447242737)]

---

## Vectorize Sentences

- Initialize tokenizer with num_words = MAX_NB_WORDS (200K). i.e. The tokenizer will perform a word count, sorted by number of occurences in descending order and pick top N words, 200K in this case 
- Use tokenizer's texts_to_sequences method to convert text to array of integers.
- The arrays obtained from previous step might not be of uniform length, use pad_sequences method to obtain arrays with length equal to MAX_SEQUENCE_LENGTH (30)

In [52]:
MAX_NB_WORDS = len(bag_of_words) #200k
MAX_SEQUENCE_LENGTH = 23                                

In [53]:
all_text = data_copy['text']
all_text = all_text.drop_duplicates (keep = False)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(all_text)

data_sequences = tokenizer.texts_to_sequences(data_copy['text'])
data_vec = pad_sequences(data_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [54]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#49421

Found 49418 unique tokens.


In [55]:
data_vec[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    1,    3,   94, 2461,
       1432])

A word_index has a unique ID assigned to each word in the data. Example:

In [56]:
word_index = tokenizer.word_index
test_string = "ropa deporte abrigo raqueta bebe"
print("word\t\tid")
print("-" * 20)
for word in test_string.split():
    print("%s\t\t%s" % (word, word_index[word]))

word		id
--------------------
ropa		34
deporte		12
abrigo		94
raqueta		1645
bebe		4


In [57]:
EMBEDDING_DIM=200

In [58]:
word_vectors = model.wv
vocabulary_size = min(len(word_index)+1,MAX_NB_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

embedding_layer = Embedding(input_dim = vocabulary_size,
                            output_dim = EMBEDDING_DIM,
                            input_length = MAX_SEQUENCE_LENGTH,
                            weights=[embedding_matrix],
                            name='w2v_embedding',
                            trainable=False)

In [59]:
model.save("w2v_embedding_v1_1.h5")

In [60]:
embedding_layer_2 = model.wv.get_keras_embedding()

In [126]:
VOCAB_SIZE = word_index
timesteps = X_train.shape[0]

In [167]:
#inp = Input(shape=(EMBEDDING_DIM, ), name = 'aaa')
input_i = Input(shape=(MAX_SEQUENCE_LENGTH, ), name='main_input')
embedded_sequences = embedding_layer_2(input_i)
encoded = LSTM(EMBEDDING_DIM, return_sequences=False)(embedded_sequences)

decoded = RepeatVector(MAX_SEQUENCE_LENGTH)(encoded)
decoded = LSTM(EMBEDDING_DIM, return_sequences=True)(decoded)
#decoded = Dense(y_train.shape[1], activation='softmax')(decoded)

sequence_autoencoder = Model(input_i, decoded)
encoder = Model(input_i, encoded)


In [160]:
print(encoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 23)                0         
_________________________________________________________________
embedding_3 (Embedding)      multiple                  2600800   
_________________________________________________________________
lstm_47 (LSTM)               (None, 200)               320800    
Total params: 2,921,600
Trainable params: 320,800
Non-trainable params: 2,600,800
_________________________________________________________________
None


In [161]:
print(sequence_autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 23)                0         
_________________________________________________________________
embedding_3 (Embedding)      multiple                  2600800   
_________________________________________________________________
lstm_47 (LSTM)               (None, 200)               320800    
_________________________________________________________________
repeat_vector_21 (RepeatVect (None, 23, 200)           0         
_________________________________________________________________
lstm_48 (LSTM)               (None, 23, 200)           320800    
Total params: 3,242,400
Trainable params: 641,600
Non-trainable params: 2,600,800
_________________________________________________________________
None


In [162]:
X_train = np.vstack(data_vec)
X_embedded = encoder.predict(X_train)
sequence_autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy')

InvalidArgumentError: indices[7,22] = 13391 is not in [0, 13004)
	 [[{{node embedding_3_12/embedding_lookup}} = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_3/embeddings/read, embedding_3_12/Cast, embedding_3_12/embedding_lookup/axis)]]

In [163]:
X_train.shape

(204812, 23)

In [165]:
X_train = np.vstack(data_vec)
sequence_autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy')
sequence_autoencoder.fit(X_train, X_train,
                         epochs=10,
                         batch_size=32,
                         verbose = 1)

ValueError: Error when checking target: expected lstm_48 to have 3 dimensions, but got array with shape (204812, 23)

In [194]:
! pip install pydot

In [189]:
X_train = np.vstack(data_vec)
X_train[0]
sequence = X_train[0].reshape((1, MAX_SEQUENCE_LENGTH, 1))
sequence

array([[[   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   1],
        [   3],
        [  94],
        [2461],
        [1432]]])

In [210]:
model = Sequential()
#model.add(embedding_layer_2)
model.add(LSTM(100, activation='relu', input_shape=(MAX_SEQUENCE_LENGTH,1)))
model.add(RepeatVector(MAX_SEQUENCE_LENGTH))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))

model.compile(optimizer='adam', loss='mse')

In [217]:
# fit model
model.fit(sequence, sequence, epochs=300, verbose=1)

Epoch 1/300
1/1 [==============================] - 0s 26ms/step - loss: 80578.4219
Epoch 2/300
1/1 [==============================] - 0s 33ms/step - loss: 80329.8672
Epoch 3/300
1/1 [==============================] - 0s 21ms/step - loss: 80076.9453
Epoch 4/300
1/1 [==============================] - 0s 24ms/step - loss: 79821.0234
Epoch 5/300
1/1 [==============================] - 0s 30ms/step - loss: 79562.1328
Epoch 6/300
1/1 [==============================] - 0s 26ms/step - loss: 79305.6719
Epoch 7/300
1/1 [==============================] - 0s 27ms/step - loss: 79036.8984
Epoch 8/300
1/1 [==============================] - 0s 26ms/step - loss: 78769.9531
Epoch 9/300
1/1 [==============================] - 0s 26ms/step - loss: 78499.8906
Epoch 10/300
1/1 [==============================] - 0s 33ms/step - loss: 79958.5312
Epoch 11/300
1/1 [==============================] - 0s 26ms/step - loss: 78043.3516
Epoch 12/300
1/1 [==============================] - 0s 33ms/step - loss: 77856.6328
E

Epoch 99/300
1/1 [==============================] - 0s 50ms/step - loss: 40708.6992
Epoch 100/300
1/1 [==============================] - 0s 22ms/step - loss: 39979.8984
Epoch 101/300
1/1 [==============================] - 0s 44ms/step - loss: 39203.9766
Epoch 102/300
1/1 [==============================] - 0s 54ms/step - loss: 38532.6367
Epoch 103/300
1/1 [==============================] - 0s 38ms/step - loss: 36677.3281
Epoch 104/300
1/1 [==============================] - 0s 24ms/step - loss: 35846.0664
Epoch 105/300
1/1 [==============================] - 0s 40ms/step - loss: 34963.7734
Epoch 106/300
1/1 [==============================] - 0s 24ms/step - loss: 34141.9062
Epoch 107/300
1/1 [==============================] - 0s 42ms/step - loss: 33272.2383
Epoch 108/300
1/1 [==============================] - 0s 32ms/step - loss: 32481.3965
Epoch 109/300
1/1 [==============================] - 0s 33ms/step - loss: 31849.8633
Epoch 110/300
1/1 [==============================] - 0s 27ms/step 

Epoch 195/300
1/1 [==============================] - 0s 26ms/step - loss: 138314.6250
Epoch 196/300
1/1 [==============================] - 0s 26ms/step - loss: 140731.3750
Epoch 197/300
1/1 [==============================] - 0s 32ms/step - loss: 136362.0000
Epoch 198/300
1/1 [==============================] - 0s 28ms/step - loss: 135393.9844
Epoch 199/300
1/1 [==============================] - 0s 27ms/step - loss: 134595.5938
Epoch 200/300
1/1 [==============================] - 0s 27ms/step - loss: 133761.9688
Epoch 201/300
1/1 [==============================] - 0s 42ms/step - loss: 132889.7188
Epoch 202/300
1/1 [==============================] - 0s 26ms/step - loss: 131983.0000
Epoch 203/300
1/1 [==============================] - 0s 37ms/step - loss: 130997.8984
Epoch 204/300
1/1 [==============================] - 0s 30ms/step - loss: 130077.2266
Epoch 205/300
1/1 [==============================] - 0s 33ms/step - loss: 129193.3359
Epoch 206/300
1/1 [==============================] - 0

Epoch 291/300
1/1 [==============================] - 0s 31ms/step - loss: 107100.0078
Epoch 292/300
1/1 [==============================] - 0s 46ms/step - loss: 107045.7031
Epoch 293/300
1/1 [==============================] - 0s 30ms/step - loss: 106991.6172
Epoch 294/300
1/1 [==============================] - 0s 25ms/step - loss: 106937.8516
Epoch 295/300
1/1 [==============================] - 0s 31ms/step - loss: 106884.3906
Epoch 296/300
1/1 [==============================] - 0s 23ms/step - loss: 106831.2031
Epoch 297/300
1/1 [==============================] - 0s 35ms/step - loss: 106778.1875
Epoch 298/300
1/1 [==============================] - 0s 27ms/step - loss: 106725.4375
Epoch 299/300
1/1 [==============================] - 0s 34ms/step - loss: 106672.8359
Epoch 300/300
1/1 [==============================] - 0s 41ms/step - loss: 106620.4453


In [218]:
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[  10.278507     4.6185694  -28.395048   -40.63671    -52.253254
  -43.38476    -53.84574    -66.415436   -63.419407   -68.5918
  -87.71953    -88.3769    -104.479904  -121.35937   -121.529175
  -97.69838    -40.30212     61.13634    219.50768    450.84137
  777.14307   1223.1322    1763.2139   ]


In [203]:
#plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')

In [ ]:
autoencoder.save('text_autoencoder_v1.h5')
#autoencoder = load_model('text_autoencoder_v1.h5')

In [ ]:
print (autoencoder.summary())

In [ ]:
data.loc[49800]